In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import glob
import matplotlib.pyplot as plt
from fastai.vision.all import *
import librosa
import librosa.display

%matplotlib inline

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
print(train.shape)
train.head()

In [ ]:
train.target.value_counts()

In [ ]:
ss = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')
print(ss.shape)
ss.head()

In [ ]:
fns = glob.glob('../input/g2net-gravitational-wave-detection/train/*/*/*/*.npy',recursive=True)

In [ ]:
path_df = pd.DataFrame()
path_df['path'] = fns
path_df['id'] = path_df['path'].apply(lambda x: x.split('/')[-1].split('.')[0])
path_df.head()

In [ ]:
train = pd.merge(train,path_df,on='id')
train.head()

In [ ]:
train1 = train.copy(deep=True)

train = train.sample(n=10000).reset_index(drop=True)

train.shape

In [ ]:
def pad_array(arr):
    h,w = arr.shape
    return np.pad(arr, [(0, 128-h), (0, 9-w)])

In [ ]:
def get_item(_):
    return train.path.values

def get_x(fn):
    arr = np.load(fn)
    #spec1 = get_spec(arr[0,:])
    #spec1 = pad_array(spec1)
    #spec2 = get_spec(arr[1,:])
    #spec2 = pad_array(spec2)
    #spec3 = get_spec(arr[2,:])
    #spec3 = pad_array(spec3)
    
    spec1 = generate_spec(arr[0,:])
    spec2 = generate_spec(arr[1,:])
    spec3 = generate_spec(arr[2,:])
    
    img = np.concatenate((spec1,spec2,spec3),axis=-1)
        
    return img

def get_y(fn):
    return int(train.loc[train['path']==fn,'target'])
    

In [ ]:
def generate_spec(arr):
    y = arr
    y = (y - y.min())/y.max()
    S = librosa.feature.melspectrogram(y=y.reshape(-1).astype('float'),n_mels=128,fmax=8000)
    S = librosa.power_to_db(S,ref=np.max)
    S = np.abs(S)
    S = (((S - S.min())/S.max())*255).astype(np.uint8)
    return S


In [ ]:
dblock = DataBlock(
    blocks=(ImageBlock(cls=PILImageBW),CategoryBlock),
    get_items=get_item,
    get_x=get_x,
    get_y=get_y, 
    splitter=RandomSplitter(valid_pct=0.20,seed=2021),
    batch_tfms=[*aug_transforms(size=64)] # Resize
)

#dblock.summary('')

In [ ]:
dls = dblock.dataloaders(Path(''), bs=512)
dls.show_batch()

In [ ]:
learn = cnn_learner(dls,resnet18,metrics=[RocAucBinary(),accuracy])

In [ ]:
learn.fit_one_cycle(5)